First, we import our dependencies:

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import sqlite3 as sql
import pandas as pd
import os

Now, we connect to the database:

In [ ]:
conn = sql.connect('database.sqlite')
cur = conn.cursor()

We create an interactive UI for viewing samples of the database and exporting its data:

In [ ]:
def get_dataframe_from_query(query, params=None):
    cur.execute(query, params or ())
    rows = cur.fetchall()
    colnames = [desc[0] for desc in cur.description]
    return pd.DataFrame(rows, columns=colnames)

def create_button(description, style=None, handler=None):
    btn = widgets.Button(description=description)
    if style:
        btn.button_style = style
    if handler:
        btn.on_click(handler)
    return btn

def display_with_navigation(buttons, data=None):
    clear_output(wait=True)
    display(widgets.VBox(buttons))
    if data is not None:
        display(data)

def export_data(df, format_type, filename):
    export_methods = {
        'CSV': lambda: df.to_csv(f'{filename}.csv', index=False),
        'Excel': lambda: df.to_excel(f'{filename}.xlsx', index=False),
        'JSON': lambda: df.to_json(f'{filename}.json', orient='records', indent=2)
    }
    
    export_methods[format_type]()
    print(f"{format_type} export completed: {filename}.{format_type.lower()}")

def show_main_menu():
    display_with_navigation([
        create_button('View sample', handler=lambda b: show_sample_submenu()),
        create_button('Export', handler=lambda b: show_export_submenu()),
        create_button('Exit', 'danger', handler=lambda b: on_exit_clicked())
    ])

def show_category_submenu(selected_category=None):
    # Get categories and create buttons
    categories = [row[0] for row in get_dataframe_from_query("SELECT DISTINCT category FROM food_inventory ORDER BY category;").values]
    
    cat_buttons = [
        create_button(cat, handler=lambda b, category=cat: show_category_submenu(selected_category=category))
        for cat in categories
    ]
    
    return_btn = create_button('Return', 'danger', handler=lambda b: show_sample_submenu())
    buttons = cat_buttons + [return_btn]
    
    data = None
    if selected_category is not None:
        data = get_dataframe_from_query(
            "SELECT * FROM food_inventory WHERE category = ?", 
            (selected_category,)
        )
    
    display_with_navigation(buttons, data)

def show_sample_submenu():
    queries = {
        'random': ("SELECT * FROM food_inventory ORDER BY RANDOM() LIMIT 5;", None),
        'expiring': ("SELECT * FROM food_inventory WHERE days_until_expiry <= 30;", None)
    }
    
    def create_data_handler(query_key):
        def handler(b):
            query, params = queries[query_key]
            df = get_dataframe_from_query(query, params)
            buttons = [
                create_button('View random sample', handler=create_data_handler('random')),
                create_button('View by category', handler=lambda b: show_category_submenu()),
                create_button('View expiring soon', handler=create_data_handler('expiring')),
                create_button('Return', 'danger', handler=lambda b: show_main_menu())
            ]
            display_with_navigation(buttons, df)
        return handler
    
    buttons = [
        create_button('View random sample', handler=create_data_handler('random')),
        create_button('View by category', handler=lambda b: show_category_submenu()),
        create_button('View expiring soon', handler=create_data_handler('expiring')),
        create_button('Return', 'danger', handler=lambda b: show_main_menu())
    ]
    
    display_with_navigation(buttons)

def show_export_submenu():
    def create_export_handler(format_type):
        def handler(b):
            df = get_dataframe_from_query("SELECT * FROM food_inventory;")
            export_data(df, format_type, 'food_inventory')
        return handler
    
    buttons = [
        create_button('Export to CSV', handler=create_export_handler('CSV')),
        create_button('Export to Excel', handler=create_export_handler('Excel')),
        create_button('Export to JSON', handler=create_export_handler('JSON')),
        create_button('Return', 'danger', handler=lambda b: show_main_menu())
    ]
    
    display_with_navigation(buttons)

def on_exit_clicked():
    menu_box.close()

# Initialize the main menu
show_main_menu()

> **Note:** In theory, I think the export functions should work, but I have a potato laptop that crashes everytime I try to run it, so I'm going to have to rely on trust for this ¯\\\_(ツ)\_/¯

Now let's visualize the data:

Finally, we close the connection to the database:

In [ ]:
conn.close()